In [1]:
from datasets import load_dataset
import pandas as pd
from feature.selector import Selective, SelectionMethod
from textwiser import TextWiser, Embedding, Transformation


/Applications/anaconda3/envs/neurips_challenge/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
args={}
args['selection_percentage'] = 0.1

In [45]:
def get_selected_data(df, args, text_column="text", label_column="category", max_clusters=2000):
    num_rows = len(df)
    df[text_column] = df["instruction"] + " " + df["input"] + " " + df["output"]
    df_T = df.loc[:, text_column].to_frame().T
    
    labels = pd.get_dummies(df[label_column], dtype=int)
    labels.columns = ["label_" + str(i) for i in range(1, len(labels.columns)+1)] 
    labels_T = labels.T
    
    # TextWiser featurization method to create text embeddings
    textwiser = TextWiser(Embedding.TfIdf(), Transformation.NMF(n_components=20))

    # Text-based selection
    # The goal is to select a subset of articles 
    # that is most diverse in the text embedding space of articles
    # and covers the most labels in each topic
    selector = Selective(SelectionMethod.TextBased(num_features=min(round(num_rows*args["selection_percentage"]), max_clusters), 
                                                   featurization_method=textwiser,
                                                   optimization_method='kmeans'))

    # Feature reduction
    subset = selector.fit_transform(df_T, labels_T)
    
    return df.loc[subset.columns, ["instruction", "input", "output"]]

In [5]:
dataset = load_dataset("conceptofmind/flan2021_submix_original", split="train")

Resolving data files: 100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 18/18 [00:00<00:00, 148.10it/s]



Generating train split: 100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 5362361/5362361 [00:09<00:00, 554356.74 examples/s]


In [6]:
df = pd.DataFrame(dataset)
df.head()

,inputs,targets,task_source,task_name,template_type
0,"Aleksandr Chumakov, club, FC Torpedo Moscow; F...",Aleksandr Chumakov has played for FC Torpedo M...,Flan2021,gem/web_nlg_en:1.1.0,zs_noopt
1,"I kill Kurtz, they kill me, the brass running ...","Já zabiju Kurtze, oni zabijou mě, hlavouni, kt...",Flan2021,wmt16_translate/cs-en:1.0.0,zs_noopt
2,i'm 10x cooler than all of you! \nWhat is the ...,positive,Flan2021,sentiment140:1.0.0,zs_noopt
3,"test: Tämä tarkoittaa, että avustuksia on jaet...","Returning to the Palestinian elections, I do n...",Flan2021,wmt16_translate/fi-en:1.0.0,fs_noopt
4,"How do you say ""In this regard, we would like ...","En este sentido, quisiéramos brindar un agrade...",Flan2021,para_crawl_enes,zs_opt


In [24]:
len(df['task_name'].unique())

70

In [26]:
len(df['template_type'].unique())

4

In [23]:
df['task_name'].value_counts()

task_name
glue/mnli:2.0.0                216560
wmt14_translate/fr-en:1.0.0    109197
trivia_qa/rc:1.1.0             109120
paws_wiki:1.1.0                109019
wmt16_translate/fi-en:1.0.0    108923
                                ...  
unified_qa_science_inst          2135
glue/wnli:2.0.0                  2111
super_glue/wsc.fixed:1.0.2       1832
super_glue/copa:1.0.2            1302
super_glue/cb:1.0.2               745
Name: count, Length: 70, dtype: int64

In [25]:
task_name_counts = df['task_name'].value_counts().to_frame()
task_name_counts.head()

,count
task_name,
glue/mnli:2.0.0,216560
wmt14_translate/fr-en:1.0.0,109197
trivia_qa/rc:1.1.0,109120
paws_wiki:1.1.0,109019
wmt16_translate/fi-en:1.0.0,108923


In [27]:
df["input"] = ""
df.rename(columns={"inputs":"instruction", "targets":"output"}, inplace=True)
df.head()

,instruction,output,task_source,task_name,template_type,input
0,"Aleksandr Chumakov, club, FC Torpedo Moscow; F...",Aleksandr Chumakov has played for FC Torpedo M...,Flan2021,gem/web_nlg_en:1.1.0,zs_noopt,
1,"I kill Kurtz, they kill me, the brass running ...","Já zabiju Kurtze, oni zabijou mě, hlavouni, kt...",Flan2021,wmt16_translate/cs-en:1.0.0,zs_noopt,
2,i'm 10x cooler than all of you! \nWhat is the ...,positive,Flan2021,sentiment140:1.0.0,zs_noopt,
3,"test: Tämä tarkoittaa, että avustuksia on jaet...","Returning to the Palestinian elections, I do n...",Flan2021,wmt16_translate/fi-en:1.0.0,fs_noopt,
4,"How do you say ""In this regard, we would like ...","En este sentido, quisiéramos brindar un agrade...",Flan2021,para_crawl_enes,zs_opt,


In [28]:
unique_task_names = list(task_name_counts.index)

In [38]:
from tqdm.notebook import tqdm

In [49]:
selected_dfs = []
for i, task_name in enumerate(unique_task_names):
    df_to_select = df[df['task_name']==task_name]
    selected_dfs.append(get_selected_data(df_to_select, args, text_column="text", 
                                          label_column="task_name", max_clusters=2000)) # label_column is not used
    print(i)
merged_df = pd.concat(selected_dfs, ignore_index=True)
merged_df.to_csv("../data/flan2021.csv", index=False)

/var/folders/32/7pkwtg797tzc2p6ftg1qgx8h0000gr/T/ipykernel_75665/85222426.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[text_column] = df["instruction"] + " " + df["input"] + " " + df["output"]


0


/var/folders/32/7pkwtg797tzc2p6ftg1qgx8h0000gr/T/ipykernel_75665/85222426.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[text_column] = df["instruction"] + " " + df["input"] + " " + df["output"]


1


/var/folders/32/7pkwtg797tzc2p6ftg1qgx8h0000gr/T/ipykernel_75665/85222426.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[text_column] = df["instruction"] + " " + df["input"] + " " + df["output"]


...

67


/var/folders/32/7pkwtg797tzc2p6ftg1qgx8h0000gr/T/ipykernel_75665/85222426.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[text_column] = df["instruction"] + " " + df["input"] + " " + df["output"]


68


/var/folders/32/7pkwtg797tzc2p6ftg1qgx8h0000gr/T/ipykernel_75665/85222426.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[text_column] = df["instruction"] + " " + df["input"] + " " + df["output"]
/Applications/anaconda3/envs/neurips_challenge/lib/python3.10/site-packages/sklearn/decomposition/_nmf.py:1665: ConvergenceWarning: Maximum number of iterations 200 reached. Increase it to improve convergence.
  warnings.warn(


69
